In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from keras.models import load_model
import pandas as pd
import numpy as np
from PIL import Image
from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, auc, f1_score
from tqdm import tqdm
import scikitplot as skplt
import glob

/home/yannis/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
test = pd.read_csv("test.csv")
test.head(2)

,lesion_id,image_id,dx,dx_type,age,sex,localization,label,label_cat
0,HAM_0001233,ISIC_0029110,nv,follow_up,35.0,male,upper extremity,0,5
1,HAM_0002312,ISIC_0025650,bcc,histo,70.0,male,face,1,1


In [3]:
x_test = []
x_test96 = []

y_test = []
y_test_cat = []
for img in tqdm(test.iterrows()):
    n = img[1]["image_id"]+".jpg"
    
    im = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-4-skin-cancer/data/all/"+n)
    image2 = im.resize((224,224))
    image3 = im.resize((96,96))

    x_test.append(preprocess_input(np.array(image2).astype(np.float32)))
    x_test96.append(preprocess_input(np.array(image3).astype(np.float32)))
    
    y_test.append(int(img[1]["label"]))
    y_test_cat.append(int(img[1]["label_cat"]))
    
x_test = np.array(x_test)
x_test96 = np.array(x_test96)
y_test = np.array(y_test)
y_test_cat = np.array(y_test_cat)

2003it [00:12, 161.46it/s]


In [4]:
def calcul_metrics(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    return (roc_auc_score(y_true, y_pred), auc(recall, precision))

## Binary predict

In [5]:
models = glob.glob("../models/dataset-4-skincancer/comparaison/*/best*")

In [7]:
for md in models:
    print(" -------- ", md.split("/")[-2], " --------- ")
    model = load_model(md, compile=False)
    if "96x96" in md:
        pred = model.predict(x_test96)
    else:
        pred = model.predict(x_test)
    roc, pr = calcul_metrics(y_test, pred[:,1])
    print("Acc : ", accuracy_score(y_test, np.argmax(pred, axis=1)))
    print("Roc :", roc)
    print("PR :", pr)

 --------  mobilnet_96x96  --------- 
Acc :  0.8801797304043935
Roc : 0.9097910174966524
PR : 0.7508233379322484
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.9151273090364453
Roc : 0.939777912459622
PR : 0.8271014686016583
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.9031452820768847
Roc : 0.9360106109549224
PR : 0.8060183913309841
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.8706939590614079
Roc : 0.8997924771375807
PR : 0.7284011021713401
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.8801797304043935
Roc : 0.9109952212625259
PR : 0.7521051720277244
 --------  Mobilnet_Pretrained_weighted  --------- 
Acc :  0.8991512730903645
Roc : 0.9365135524487063
PR : 0.8120342787448722
 --------  Mobilnet_Pretrained_weighted  --------- 
Acc :  0.19520718921617575
Roc : 0.5249011569240923
PR : 0.5052627657891682
 --------  mobilnet_224x224  --------- 
Acc :  0.9041437843235147
Roc : 0.937852614343828
PR : 0.817720790374242
 --------  mobilnet_224x224_weighted

## Cat predict

In [8]:
models = glob.glob("../models/dataset-4-skincancer/comparaison-cat/*/best*")

In [9]:
for md in models:
    print(" -------- ", md.split("/")[-2], " --------- ")
    model = load_model(md, compile=False)
    if "96x96" in md:
        pred = model.predict(x_test96)
    else:
        pred = model.predict(x_test)
#     roc, pr = calcul_metrics(y_test_cat, pred[:,1])
    print("Acc : ", accuracy_score(y_test_cat, np.argmax(pred, axis=1)))
#     print("Roc :", roc)
#     print("PR :", pr)

 --------  mobilnet_96x96  --------- 
Acc :  0.8337493759360959
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.8956565152271593
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.8881677483774338
 --------  Mobilnet_GAN-labelised  --------- 
Acc :  0.7363954068896655
 --------  mobilnet_224x224  --------- 
Acc :  0.8786819770344483
 --------  mobilnet_224x224_weighted  --------- 
Acc :  0.8836744882675986
 --------  Mobilnet_ACGAN_weighted  --------- 
Acc :  0.9036445332001997
 --------  mobilnet_96x96_weighted  --------- 
Acc :  0.8247628557164254


# ------ Old predict --------

In [48]:
x_test = []
x_test_2 = []
y_test = []
for img in tqdm(test.iterrows()):
    n = img[1]["image_id"]+".jpg"
    
#     im = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-4-skin-cancer/data/all/"+n)
#     image = im.resize((224,224))
    
    im = Image.open("/home/yannis/Developpement/ppd-GAN-for-medical-imaging/dataset-4-skin-cancer/data/all_600_450/"+n)
    image2 = im.crop((75, 0, 525, 450)).resize((224,224))
    
#     x_test.append(preprocess_input(np.array(image)))
    x_test_2.append(preprocess_input(np.array(image2).astype(np.float32)))
    
    y_test.append(int(img[1]["cell_type_idx"]))
    
# x_test = np.array(x_test)
x_test_2 = np.array(x_test_2)
y_test = np.array(y_test)

1703it [00:23, 71.15it/s]


### MobileNetV2_224x224_weight_600x450_2

In [54]:
model4 = load_model("../models/dataset-4-skincancer/MobileNetV2_224x224_weight_600x450_2/best_model_epoch.hdf5")

In [55]:
pred4 = model4.predict(x_test_2)

In [56]:
accuracy_score(y_test, np.argmax(pred4, axis=1))

0.8807985907222549

### MobileNetV2_224x224_weight_600x450

In [37]:
model3 = load_model("../models/dataset-4-skincancer/MobileNetV2_224x224_weight_600x450/best_model_epoch.hdf5")

In [38]:
pred3 = model1.predict(x_test_2)

In [39]:
accuracy_score(y_test, np.argmax(pred3, axis=1))

0.8708162066940693

In [43]:
# skplt.metrics.plot_confusion_matrix(y_test, np.argmax(pred3, axis=1))

### MobileNetV2_224x224_weight

In [31]:
model1 = load_model("../models/dataset-4-skincancer/MobileNetV2_224x224_weight/best_model_epoch.hdf5")

In [32]:
pred1 = model1.predict(x_test)

In [33]:
accuracy_score(y_test, np.argmax(pred1, axis=1))

0.8790369935408103

### MobileNetV2_224x224_2

In [34]:
model2 = load_model("../models/dataset-4-skincancer/MobileNetV2_224x224_2/best_model_epoch.hdf5")

In [35]:
pred2 = model2.predict(x_test)

In [36]:
accuracy_score(y_test, np.argmax(pred2, axis=1))

0.8590722254844392